In [3]:
import numpy as np
import pandas as pd

Tags = pd.read_csv("Tags.csv")
Posts = pd.read_csv("Posts.csv")
Users = pd.read_csv("Users.csv")
Votes = pd.read_csv("Votes.csv")
Comments = pd.read_csv("Comments.csv")
Badges = pd.read_csv("Badges.csv")

import sqlite3
con = sqlite3.connect("./travel.db")
Tags.to_sql("Tags", con)
Posts.to_sql("Posts", con)
Users.to_sql("Users", con)
Votes.to_sql("Votes", con)
Comments.to_sql("Comments", con)
Badges.to_sql("Badges", con)

## Zadanie1  
>Zwraca 10 użytkowników, których pytania zostały w sumie dodane do ulubionch największą ilość razy.  
>Zwraca nazwę, id, lokację użytkownika, sumę dodań do ulubionych dla wszystkich zadanych przez niego pytań,
>pytanie tego użytkownika dodane do ulubionych największą liczbę razy i liczbę dodań do ulubionych dla tego pytania.  

In [67]:
sql = pd.read_sql_query("""SELECT Users.DisplayName,
                           Users.Id,
                           Users.Location,
                           SUM(Posts.FavoriteCount) AS FavoriteTotal,
                           Posts.Title AS MostFavoriteQuestion,
                           MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                           FROM Posts
                           JOIN Users ON Users.Id=Posts.OwnerUserId
                           WHERE Posts.PostTypeId=1
                           GROUP BY OwnerUserId
                           ORDER BY FavoriteTotal DESC
                           LIMIT 10""", con)

pandas = Users[['DisplayName', 'Id', 'Location']].set_index('Id').join(\
Posts.query("PostTypeId == 1")[['Title', 'FavoriteCount', 'OwnerUserId']].\
set_index('OwnerUserId'), how = 'inner').sort_values('FavoriteCount', ascending = False).groupby(level = 0).\
agg({'DisplayName': 'first', 'Location': 'first', 'FavoriteCount': ['sum', 'max'], 'Title': 'first'})
pandas.columns = pandas.columns.droplevel(0)
pandas = pandas.sort_values('sum', ascending = False).head(10).reset_index().iloc[:,[1,0,2,3,5,4]]
pandas.columns = sql.columns

sql.equals(pandas)

True

## Zadanie2
>Zwraca 10 pytań (id, tytuł, liczba odpowiedzi o dodatnim wyniku na dane pytanie)  
>z największą liczbą odpowiedzi o dodatnim wyniku na dane pytanie. 

In [99]:
sql = pd.read_sql_query(""" SELECT Posts.ID,
                            Posts.Title,
                            Posts2.PositiveAnswerCount
                            FROM Posts
                            JOIN (
                            SELECT
                            Posts.ParentID,
                            COUNT(*) AS PositiveAnswerCount
                            FROM Posts
                            WHERE Posts.PostTypeID=2 AND Posts.Score>0
                            GROUP BY Posts.ParentID
                            ) AS Posts2
                            ON Posts.ID=Posts2.ParentID
                            ORDER BY Posts2.PositiveAnswerCount DESC
                            LIMIT 10""", con)

pandas = Posts.query("PostTypeId == 2 and Score > 0").groupby('ParentId').size().nlargest(10).to_frame().\
join(Posts.set_index('Id'), how = 'inner').iloc[:,[15,0]].reset_index()                                                                                                
pandas.columns = sql.columns

sql.equals(pandas)

True

## Zadanie3
>Dla każdego roku zwraca tytuł pytania, które dostało najwięcej upvotów w danym roku, ten rok i tę liczbę upVotów. 

In [187]:
sql = pd.read_sql_query(""" SELECT Posts.Title,
                            UpVotesPerYear.Year,
                            MAX(UpVotesPerYear.Count) AS Count
                            FROM (
                            SELECT
                            PostId,
                            COUNT(*) AS Count,
                            STRFTIME('%Y', Votes.CreationDate) AS Year
                            FROM Votes
                            WHERE VoteTypeId=2
                            GROUP BY PostId, Year
                            ) AS UpVotesPerYear
                            JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
                            WHERE Posts.PostTypeId=1
                            GROUP BY Year""", con)

q_uv_y = Votes.query("VoteTypeId == 2")[['PostId', 'CreationDate']]
q_uv_y.CreationDate = q_uv_y.CreationDate.str[:4]
q_uv_y = q_uv_y.groupby(['PostId', 'CreationDate']).size().to_frame().join(\
Posts.query("PostTypeId == 1")[['Id', 'Title']].set_index('Id'), on = 'PostId', how = 'inner')
pandas = q_uv_y.groupby('CreationDate').max()[0].to_frame().reset_index().set_index(['CreationDate', 0]).join(\
q_uv_y.reset_index().set_index(['CreationDate', 0])).reset_index().iloc[:,[3,0,1]]
pandas.columns = sql.columns

sql.equals(pandas)

True

## Zadanie4
>Zwraca pytania z największą różnicą pomiędzy wynikiem najwyżej punktowanej i zaakceptowanej odpowiedzi,  
>posortowane malejąco względem tej różnicy. 

In [232]:
sql = pd.read_sql_query(""" SELECT Questions.Id,
                            Questions.Title,
                            BestAnswers.MaxScore,
                            Posts.Score AS AcceptedScore,
                            BestAnswers.MaxScore-Posts.Score AS Difference
                            FROM (
                            SELECT Id, ParentId, MAX(Score) AS MaxScore
                            FROM Posts
                            WHERE PostTypeId==2
                            GROUP BY ParentId
                            ) AS BestAnswers
                            JOIN (
                            SELECT * FROM Posts
                            WHERE PostTypeId==1
                            ) AS Questions
                            ON Questions.Id=BestAnswers.ParentId
                            JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
                            WHERE Difference>50
                            ORDER BY Difference DESC""", con)

a = Posts.query("PostTypeId == 2")[['Id', 'ParentId', 'Score']]
aa = a.iloc[:,[0,2]]
aa.columns = ['Id', 'AccScore']
pandas = a.groupby('ParentId').max().join(Posts.query("PostTypeId == 1 and AcceptedAnswerId == AcceptedAnswerId")\
[['Id', 'Title', 'AcceptedAnswerId']].set_index('Id'), how = 'inner').join(aa.set_index('Id'), on = 'AcceptedAnswerId')
pandas['Dif'] = pandas['Score'] - pandas['AccScore']
pandas = pandas.query("Dif > 50").sort_values('Dif', ascending = False).reset_index().iloc[:,[0,3,2,5,6]]
pandas.columns = sql.columns

sql.equals(pandas)

True

## Zadanie5
>Zwraca 10 pytań (tytuł pytania, *wynik komentarzy autora*) z największym *wynikiem komentarzy autora*,  
>gdzie *wynik komentarzy autora* to suma punktów uzyskanych przez autora pytania w komentarzach do tego pytania. 

In [258]:
sql = pd.read_sql_query(""" SELECT Posts.Title,
                            CmtTotScr.CommentsTotalScore
                            FROM (
                            SELECT
                            PostID,
                            UserID,
                            SUM(Score) AS CommentsTotalScore
                            FROM Comments
                            GROUP BY PostID, UserID
                            ) AS CmtTotScr
                            JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
                            WHERE Posts.PostTypeId=1
                            ORDER BY CmtTotScr.CommentsTotalScore DESC
                            LIMIT 10""", con)

pandas = Comments.groupby(['PostId', 'UserId']).sum().join(Posts.query("PostTypeId == 1")[['Id', 'Title', 'OwnerUserId']].\
set_index(['Id', 'OwnerUserId']), on = ['PostId', 'UserId'], how = 'inner').reset_index().iloc[:,[4,3]].\
nlargest(10, 'Score').reset_index(drop = True)
pandas.columns = sql.columns

sql.equals(pandas)

True

## Zadanie6
>Zwraca użytkowników (id, nazwę, reputację, wiek, lokację), którzy są w posiadaniu *wartościowej* odznaki.  
>Odznaka jest *wartościowa* wtedy i tylko wtedy gdy jest <span style="color:gold">złota</span> i została zdobyta przez od 2 do 10 użytkowników.  


In [328]:
sql = pd.read_sql_query(""" SELECT DISTINCT Users.Id,
                            Users.DisplayName,
                            Users.Reputation,
                            Users.Age,
                            Users.Location
                            FROM (
                            SELECT
                            Name, UserID
                            FROM Badges
                            WHERE Name IN (
                            SELECT
                            Name
                            FROM Badges
                            WHERE Class=1
                            GROUP BY Name
                            HAVING COUNT(*) BETWEEN 2 AND 10
                            )
                            AND Class=1
                            ) AS ValuableBadges
                            JOIN Users ON ValuableBadges.UserId=Users.Id""", con)

b = Badges.query("Class == 1")[['Name', 'UserId']]
vbNames = b.groupby('Name').size().between(2,10)
vbNames = vbNames[vbNames].index

vbUid = b[b['Name'].isin(vbNames)]['UserId']
pandas = Users[Users['Id'].isin(vbUid)][['Id', 'DisplayName', 'Reputation', 'Age', 'Location']].reset_index(drop = True)

sql.sort_values(id).equals(pandas)

False

In [262]:
b = Badges.query("Class == 1")[['Name', 'UserId']]

In [326]:
vbNames = b.groupby('Name').size().between(2,10)
vbNames = vbNames[vbNames].index

vbUid = b[b['Name'].isin(vbNames)]['UserId']
pandas = Users[Users['Id'].isin(vbUid)][['Id', 'DisplayName', 'Reputation', 'Age', 'Location']].reset_index(drop = True)



In [327]:
pandas

,Id,DisplayName,Reputation,Age,Location
0,19,VMAtm,18556,33.0,"Tampa, FL, United States"
1,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
2,108,Ankur Banerjee,31273,27.0,"London, UK"
3,466,iHaveacomputer,8360,NaN,Down underer
4,693,RoflcoptrException,33300,NaN,NaN
5,793,mindcorrosive,10531,32.0,Bulgaria
6,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
7,6669,Relaxed,69405,NaN,NaN
8,19400,phoog,34342,50.0,"New York, NY"
9,39065,Pont,1004,NaN,Austria


In [260]:
sql

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,None
6,6669,Relaxed,69405,NaN,None
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


## Zadanie7


In [ ]:
sql = pd.read_sql_query("""SELECT Users.DisplayName,
                           Users.Id,
                           Users.Location,
                           SUM(Posts.FavoriteCount) AS FavoriteTotal,
                           Posts.Title AS MostFavoriteQuestion,
                           MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                           FROM Posts
                           JOIN Users ON Users.Id=Posts.OwnerUserId
                           WHERE Posts.PostTypeId=1
                           GROUP BY OwnerUserId
                           ORDER BY FavoriteTotal DESC
                           LIMIT 10""", con)

pandas = Users[['DisplayName', 'Id', 'Location']].set_index('Id').join(\
Posts.query("PostTypeId == 1")[['Title', 'FavoriteCount', 'OwnerUserId']].\
set_index('OwnerUserId'), how = 'inner').sort_values('FavoriteCount', ascending = False).groupby(level = 0).\
agg({'DisplayName': 'first', 'Location': 'first', 'FavoriteCount': ['sum', 'max'], 'Title': 'first'})
pandas.columns = pandas.columns.droplevel(0)
pandas = pandas.sort_values('sum', ascending = False).head(10).reset_index().iloc[:,[1,0,2,3,5,4]]
pandas.columns = sql.columns

sql.equals(pandas)

![pandas](https://user-images.githubusercontent.com/43205483/58571663-830ea580-823a-11e9-9f0f-03925c92cefe.png)